In [5]:
import requests
import pandas as pd
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os
import geopandas as gpd

In [6]:
# Read data

os.chdir('C:/Users/6997619/Documents/GitHub projects/DHZW_routing/data')

df_PC6_Moerwijk = gpd.read_file('C:/Users/6997619/Documents/GitHub projects/DHZW_shapefiles/data/processed/shapefiles/df_PC6_station_shp/df_PC6_station_shp.shp')
df_PC6_DHZW = gpd.read_file('C:/Users/6997619/Documents/GitHub projects/DHZW_shapefiles/data/processed/shapefiles/PC6_DHZW_shp/PC6_DHZW_shp.shp')
df = pd.read_csv('OD_asymmetric.csv')

In [7]:
# Set up variables
df['time_total'] = np.nan
df['time_walk'] = np.nan
df['time_transit_bus'] = np.nan
df['time_transit_train'] = np.nan
df['time_waiting'] = np.nan
df['n_changes'] = np.nan
df['distance_total'] = np.nan
df['distance_walk'] = np.nan
df['distance_bus'] = np.nan
df['distance_train'] = np.nan
df['stop_PC6'] = np.nan

# Filter out trips that are completely inside or outside DHZW
df = df[(df['departure_in_DHZW'] == 1) & (df['arrival_in_DHZW'] == 0) |
        (df['departure_in_DHZW'] == 0) & (df['arrival_in_DHZW'] == 1)]


In [27]:
# Function to call the OpenTripPlanner REST API
def call_otp(from_x, from_y, to_x, to_y):
    base_url = "http://localhost:8801/otp/routers/default/plan?"
    params = {
        "fromPlace": f"{from_y},{from_x}",
        "toPlace": f"{to_y},{to_x}",
        "mode": 'WALK,BUS,TRAM,SUBWAY',
        "date": "2023-02-09",
        "time": "08:00:00",
        "arriveBy": "false",
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        return response.json()
    except RequestException as e:
        print(e)
        return None

In [28]:
def find_bus_stop_within_DHZW(route, found_train_via_Moerwijk):
    found = False
    # If the train goes through Moerwijk, return the Moerwijk bus stop
    if found_train_via_Moerwijk:
        return '2532CP'
    else:
        # Look for the bus stop within DHZW
        for _, leg in route.iterrows():
            if leg['leg_mode'] in ['BUS', 'TRAM', 'SUBWAY']:
                line = LineString(leg['geometry']['coordinates'])
                # Retrieve start point of the leg
                point_1 = Point(line.coords[0])
                # Find if there is one of the postcodes of the DHZW matches with the departure or arrival
                if df_PC6_DHZW.intersects(point_1).any():
                    found = True
                    break
        if found:
            return df_PC6_DHZW_postcodes.iloc[0]['PC6']
        else:
            return np.nan

In [31]:
for i in range(1, 2):
    print(round(((i/df.shape[0])*100), 2))

    otp_data = call_otp(str(df.loc[i-1, 'departure_x']), str(df.loc[i-1, 'departure_y']), str(df.loc[i-1, 'arrival_x']), str(df.loc[i-1, 'arrival_y']))
    itineraries = otp_data['plan']['itineraries']
            
    # check if there are at least two itineraries. if it was only foot, there would be only one itinerary
    if len(itineraries) > 1:
        # filter out itineraries that are only by foot
        itineraries = [it for it in itineraries if 'WALK' not in [leg['mode'] for leg in it['legs']]]

        print(itineraries)

            
        # select the fastest option
        fastest_itinerary = min(itineraries, key=lambda it: it['duration'])
        
        
        # filter the faster route
        legs = fastest_itinerary['legs']
        legs_data = []
        for leg in legs:
            legs_data.append({
                'fromPlace': leg['from']['name'],
                'toPlace': leg['to']['name'],
                'duration': leg['duration']/60,
                'distance': leg['distance']/1000,
                'mode': leg['mode'],
                'route': leg['route'] if 'route' in leg else '',
                'route_short_name': leg['routeShortName'] if 'routeShortName' in leg else '',
                'start_time': leg['startTime']
            })

        time_total = round(fastest_itinerary['duration']/60, 1)
        time_walk = round(fastest_itinerary['walkTime']/60, 1)
        time_transit = round(fastest_itinerary['transitTime']/60, 1)
        time_waiting = round(fastest_itinerary['waitingTime']/60, 1)
        n_changes = len(fastest_itinerary['legs']) - 1

        distance_total = round(sum([leg['distance'] for leg in legs])/1000, 1)
        distance_walk = round(fastest_itinerary['walkDistance']/1000, 1)
        distance_transit = round(distance_total - distance_walk, 1)

        df.at[i-1, 'time_total'] = time_total
        df.at[i-1, 'time_walk'] = time_walk
        df.at[i-1, 'time_transit'] = time_transit
        df.at[i-1, 'time_waiting'] = time_waiting

        df.at[i-1, 'n_changes'] = n_changes

0.01
[]


ValueError: min() arg is an empty sequence